In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def LabelEncoder(df): 
    encoder = dict(zip(df.unique(), range(df.nunique())))

    for index, row in df.items():
        df.at[index] = encoder[row]
        
    return df

In [3]:
def load_data(filename, columns, le): # Split data 90% training 10% testing and label encode
    df = pd.read_csv(filename, sep=',', usecols = columns, index_col=False)
    df = df.dropna() 
    
    for column in le: 
        df[column] = LabelEncoder(df[column])
        
    train_x = np.array(df[columns[:-1]])[:int(len(df[columns[:-1]])*0.9)]
    test_x = np.array(df[columns[:-1]])[-int(len(df[columns[:-1]])*0.1):]
    train_y = np.array(df[columns[-1]])[:int(len(df[columns[-1]])*0.9)]
    test_y = np.array(df[columns[-1]])[-int(len(df[columns[-1]])*0.1):]

    return train_x, train_y, test_x, test_y

In [4]:
class Node(): 

    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        
        # Only defined in leaf nodes
        self.value = value

In [47]:
class Tree:

    def __init__(self, max_depth=6):
        # Root node
        self.root = None
        
        # Branch count
        self.max_depth = max_depth
    
    def __grow(self, x, y, current_depth=0): # Recursive function, builds the tree.
        samples, depth = x.shape
        
        if current_depth >= self.max_depth or len(np.unique(y)) <= 1:
            return Node(value=np.argmax(np.bincount(y.flatten())))
        
        best_feature, best_threshold = self.__split(x, y, samples, depth)

        left_indx = np.argwhere(x[:, best_feature] <= best_threshold).flatten()
        right_indx = np.argwhere(x[:, best_feature] > best_threshold).flatten()
        
        left = self.__grow(x[left_indx, :], y[left_indx], current_depth+1)
        right = self.__grow(x[right_indx, :], y[right_indx], current_depth+1)

        return Node(best_feature, best_threshold, left, right)
        
    def __split(self, x, y, samples, features): # Finds the best split, returns the best feature (index), and optimal threshold.
        best_gain = 0
        best_threshold, best_feature_indx = None, None
        
        for feature_indx in range(features):    
            feature = x[:, feature_indx]
            thresholds = np.unique(feature)
            
            for threshold in thresholds:
                left = np.argwhere(feature <= threshold).flatten()
                right = np.argwhere(feature > threshold).flatten()
                
                gain = self.__gain(y, y[left], y[right])
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature_indx = feature_indx
                    best_threshold = threshold
        
        return best_feature_indx, best_threshold
    
    def __gain(self, parent, left, right): # Calculates the optimal information gain.
        return self.__gini_impurity(parent) - ((len(left)/ len(parent)) * self.__gini_impurity(left) + (len(right) / len(parent)) * self.__gini_impurity(right))     
    
    def __gini_impurity(self, y): # The sum of the squared probability of samples that belong to a node.
        options = np.unique(y)
        gini = 0
        for option in options:
            option_probability = len(y[y == option]) / len(y)
            gini += option_probability**2
        return 1 - gini
    
    def __traverse(self, x, node): # Checks if leaf, if not then traverses tree
        if node.value is not None:
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self.__traverse(x, node.left)
        return self.__traverse(x, node.right)
    
    def fit(self, x, y):
        self.root = self.__grow(x, y)
    
    def predict(self, x):
        return np.array([self.__traverse(i, self.root) for i in x])

In [51]:
def accuracy_score(test_y, predicted_y):
    return sum([x == y for (x, y) in zip(predicted_y, test_y)]) / len(test_y)

In [57]:
def confusion_matrix(test_y, predicted_y):
    size = len(np.unique(test_y))  
    result = np.zeros((size, size))

    for i in range(len(test_y)):
        result[test_y[i]][predicted_y[i]] += 1
        
    return result

In [58]:
if __name__ == '__main__' and '__file__' not in globals():
    train_x, train_y, test_x, test_y = load_data('data.csv', ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived'], ['Sex', 'Embarked'])
    
    clf = Tree()
    clf.fit(train_x, train_y)
    
    predicted_y = clf.predict(test_x)
    print(accuracy_score(test_y, predicted_y))
    print(confusion_matrix(test_y, predicted_y))

0.8450704225352113
[[42.  3.]
 [ 8. 18.]]
